In [ ]:
!pip install torchvision==0.15.2
!pip install -q basicsr facexlib gfpgan
!pip install -q tensorflow opencv-python torchvision


In [ ]:
import numpy as np
import cv2

In [ ]:
!pip install rarfile
!apt-get install unrar


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
import torch
from skimage.metrics import structural_similarity as ssim
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
import matplotlib.pyplot as plt
from imageio import imread
import cv2
from PIL import Image

In [ ]:
import rarfile
import re
import os

# Path to your uploaded RAR
rar_path = "forcepowers.rar"  # change to your filename

# Extract RAR
rf = rarfile.RarFile(rar_path)
rf.extractall("forcepowers")

power_names = []

# Walk through all Lua files and extract POWER.Name
for root, dirs, files in os.walk("forcepowers"):
    for filename in files:
        if filename.endswith(".lua"):
            filepath = os.path.join(root, filename)
            with open(filepath, encoding="utf-8", errors="ignore") as f:
                content = f.read()
                match = re.search(r'POWER\.Name\s*=\s*"([^"]+)"', content)
                if match:
                    power_names.append(match.group(1))

# Remove duplicates and sort alphabetically
power_names = sorted(set(power_names))

# Show the list
for name in power_names:
    print(name)

Downscale - shrinks  1/4 of its original width and height

In [ ]:
img= Image.open('/content/sa.jpg').convert('RGB')
imgnp = np.array(img)
h , w = imgnp.shape[:2]
print(h,w)
lowres = cv2.resize(imgnp, (w//4, h//4), interpolation=cv2.INTER_AREA)
lowresp = Image.fromarray(lowres)
lowresp.save('/content/lowres.png')
from imageio import imread
img = imread('/content/lowres.png', pilmode='L')

BICUBIC - Bicubic interpolation looks at a 4×4 grid of surrounding pixels (16 total) and uses cubic polynomial curves instead of linear ones to estimate new pixel values. It applies 2D cubic interpolation by first interpolating in the X direction, then in the Y direction for smoother, more natural upscaling.


If the distance is less than or equal to 1, it uses a smooth cubic curve,If the distance is between 1 and 2, it uses a different cubic equation with smaller influence.If the distance is greater than 2, the pixel has no effect (weight = 0)

In [ ]:
from imageio import imread
img = imread('/content/lowres.png' )

def cubic_interp(x, a=-0.5):
    abs_x = np.abs(x)
    abs_x2 = abs_x ** 2
    abs_x3 = abs_x ** 3

    return np.where(abs_x <= 1,
        (a + 2) * abs_x3 - (a + 3) * abs_x2 + 1,
        np.where(abs_x <= 2,
            a * abs_x3 - 5 * a * abs_x2 + 8 * a * abs_x - 4 * a,
            0))

def get_pixel(img, x, y, c):
    x = min(max(x, 0), img.shape[1] - 1)
    y = min(max(y, 0), img.shape[0] - 1)
    return img[y, x, c]

def bicubic_resize_rgb(img, scale):
    h, w, c = img.shape
    new_h = int(h * scale)
    new_w = int(w * scale)
    output = np.zeros((new_h, new_w, c), dtype=np.float32)

    for channel in range(c):
        for i in range(new_h):
            for j in range(new_w):
                x = j / scale
                y = i / scale
                x_int = int(x)
                y_int = int(y)
                dx = x - x_int
                dy = y - y_int

                value = 0.0
                for m in range(-1, 3):
                    for n in range(-1, 3):
                        px = get_pixel(img, x_int + n, y_int + m, channel)
                        wx = cubic_interp(n - dx)
                        wy = cubic_interp(m - dy)
                        value += px * wx * wy

                output[i, j, channel] = np.clip(value, 0, 255)

    return output.astype(np.uint8)
scaled = bicubic_resize_rgb(img, scale=4)
plt.imsave('/content/bicubic.png', scaled)

Nearest-Neighbor Interpolation - Just copies the nearest pixel value from original to new location.


In [ ]:
img = imread('/content/lowres.png')

def nn_resize_rgb(img, scale):
    h, w, c = img.shape
    new_h = int(h * scale)
    new_w = int(w * scale)
    output = np.zeros((new_h, new_w, c), dtype=img.dtype)

    for i in range(new_h):
        for j in range(new_w):
            src_x = int(j / scale)
            src_y = int(i / scale)
            output[i, j] = img[src_y, src_x]

    return output

scaled_nn = nn_resize_rgb(img, scale=4)

plt.imsave('/content/nn.png', scaled_nn, cmap='gray')


**Bilinear interpolation** looks at the 4 closest pixels around a new spot and blends their colors to fill it in. It makes the image smoother than just copying pixels, but not as sharp as more advanced methods.

In [ ]:
def bilinear_resize_rgb(img, scale):
    h, w, c = img.shape
    new_h = int(h * scale)
    new_w = int(w * scale)
    output = np.zeros((new_h, new_w, c), dtype=np.float32)

    for channel in range(c):
        for i in range(new_h):
            for j in range(new_w):
                x = j / scale
                y = i / scale

                x0 = int(np.floor(x))
                x1 = min(x0 + 1, w - 1)
                y0 = int(np.floor(y))
                y1 = min(y0 + 1, h - 1)

                Ia = img[y0, x0, channel]
                Ib = img[y1, x0, channel]
                Ic = img[y0, x1, channel]
                Id = img[y1, x1, channel]

                wa = (x1 - x) * (y1 - y)
                wb = (x1 - x) * (y - y0)
                wc = (x - x0) * (y1 - y)
                wd = (x - x0) * (y - y0)

                output[i, j, channel] = wa * Ia + wb * Ib + wc * Ic + wd * Id

    return np.clip(output, 0, 255).astype(np.uint8)


scaled_rgb = bilinear_resize_rgb(img, scale=4)
plt.imsave('/content/bilinear.png', scaled_rgb)

REAL ESRGAN

In [ ]:
model = RRDBNet(num_in_ch=3 , num_out_ch=3 ,num_feat= 64, num_block=23, num_grow_ch=32 , scale = 4)
upscaler = RealESRGANer(
    scale=4,
    model_path='/content/Real-ESRGAN/weights/RealESRGAN_x4plus.pth',
    model=model)
sr_img_tuple = upscaler.enhance(np.array(lowresp), outscale=4)
sr_img = sr_img_tuple[0] # Extract the image data from the tuple
sr_img = Image.fromarray(sr_img)
sr_img.save('/content/sr_img.png')

SRCNN

In [ ]:
def build_srcnn():
    return Sequential([
        Conv2D(128, (9, 9), activation='relu', padding='valid', input_shape=(None, None, 1)),
        Conv2D(64, (3, 3), activation='relu', padding='valid'),
        Conv2D(1, (5, 5), activation='linear', padding='valid')
    ])

srcnn = build_srcnn()

weights_url = 'https://raw.githubusercontent.com/MarkPrecursor/SRCNN-keras/master/3051crop_weight_200.h5'
weight_path = tf.keras.utils.get_file('3051crop_weight_200.h5', weights_url)
srcnn.load_weights(weight_path)

In [ ]:
img = cv2.imread('/content/lowres.png', cv2.IMREAD_GRAYSCALE)
h, w = img.shape
bicubic_up = cv2.resize(img, (w*2, h*2), interpolation=cv2.INTER_CUBIC)

input_img = bicubic_up.astype(np.float32) / 255.0
input_img = np.expand_dims(input_img, axis=(0, -1))
pred = srcnn.predict(input_img)
output = np.clip(pred[0, :, :, 0] * 255.0, 0, 255).astype(np.uint8)
cv2.imwrite('/content/srcnn_upscaled.png', output)

SIMILARITY SCORE


In [ ]:
def calculate_ssim(original_image_path, generated_image_path):

    original_img = cv2.imread(original_image_path)
    generated_img = cv2.imread(generated_image_path)


    if original_img.shape != generated_img.shape:

        generated_img = cv2.resize(generated_img, (original_img.shape[1], original_img.shape[0]))

        if original_img.shape[-1] != generated_img.shape[-1]:
             generated_img = cv2.cvtColor(generated_img, cv2.COLOR_BGR2RGB)


    gray_original = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    gray_generated = cv2.cvtColor(generated_img, cv2.COLOR_BGR2GRAY)


    ssim_score, _ = ssim(gray_original, gray_generated, full=True)

    return ssim_score

original_image_path = '/content/images.jpg'
bicubic_image_path = '/content/bicubic.png'
realesrgan_image_path = '/content/sr_img.png'

ssim_bicubic = calculate_ssim(original_image_path, bicubic_image_path)
ssim_realesrgan = calculate_ssim(original_image_path, realesrgan_image_path)

print(f"SSIM between original and bicubic: {ssim_bicubic:.4f}")
print(f"SSIM between original and Real-ESRGAN: {ssim_realesrgan:.4f}")
